In [3]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import time

class keydefaultdict(defaultdict):
    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError( key )
        else:
            ret = self[key] = self.default_factory(key)
            return ret


start = time.time()
commands = open('inputs/day17.txt').read().strip()

# relative positions of the squares in a rock to the bottom left square. 
# in X, Y order, Y up is negative
rock_types = [
    # dash 
    [(0, 0), (1, 0), (2, 0), (3, 0)], 
    # plus
    [(1, 0), (0, -1), (1, -1), (2, -1), (1, -2)],
    # inverse L
    [(0, 0), (1, 0), (2, 0), (2, -1), (2, -2)],
    # block 
    [(0,0), (0, -1), (0, -2), (0, -3)],
    # square
    [(0, 0), (1, 0), (0, -1), (1, -1)],
]
ROCK_BLOCK = 1
WALL_BLOCK = 2

def get_start_position(rock, tower):
    y_positions = [y for x, y in tower.keys() if tower[(x, y)] == ROCK_BLOCK]
    if y_positions:
        min_y = min(y_positions)
    else: 
        min_y = 0

    relative_x = 2
    relative_y = min_y - 4
    return [(x + relative_x, y + relative_y) for x, y in rock]

def visualise_tower(tower, rock):    
    vis_top = 10

    y_positions = [y for x, y in tower.keys() if tower[(x, y)] == ROCK_BLOCK]
    if y_positions:
        min_y = min(y_positions) - vis_top
    else: 
        min_y = 0 - vis_top

    # Create a numpy array to visualise
    vis = np.zeros((abs(min_y), 9), dtype=np.float32)
    for x, y in list(tower.keys()):
        if y < min_y:
            continue
        color = tower[(x, y)] / 2.0 

        x = x + 1
        y = y + abs(min_y) - 1

        # print(x, y, color)
        vis[y, x] = color
    
    for x, y in rock:
        x = x + 1
        y = y + abs(min_y) - 1
        vis[y, x] = 2.0
    plt.imshow(vis, cmap='gray')
    plt.show()

def height_max_block(tower):         
    a = [y for x, y in tower.keys() if tower[(x, y)] == 1]
    if a:
        return abs(min(a))
    else:
        return 0

def move_sideways(rock, command, tower):     
    # Sideways movement
    new_pos = None
    if command == '<':
        new_pos = [(x - 1, y) for x, y in rock]
    elif command == '>':
        new_pos = [(x + 1, y) for x, y in rock]
    else: 
        raise ValueError("Command does not exist")
    
    # See if there are overlaps
    overlaps = False
    for x, y in new_pos:
        if tower[(x, y)]:
            overlaps = True
            break
    
    # Move if there are no overlaps
    if not overlaps:
        rock = new_pos
    return rock


def create_empty_tower(max_height_wall = -10000):
    tower = defaultdict(int)
    # Create a floor
    for x in range(7):
        tower[(x, 0)] = WALL_BLOCK

    # Create the walls
    for y in range(max_height_wall, 0):
        tower[(-1, y)] = WALL_BLOCK
        tower[(7, y)] = WALL_BLOCK
    return tower

def move_downwards(rock, tower):
    # Downward movement
    new_pos = [(x, y + 1) for x, y in rock]

    # See if there are overlaps
    overlaps = False
    for x, y in new_pos:
        if tower[(x, y)]:
            overlaps = True
            break

    # See if we can actually move the rock down
    if not overlaps:
        rock = new_pos
    return rock, overlaps

# Store the height after each iteration as a tuple with 
# (which block last delivered, which command last executed, max height of a block)
height_after = dict()
height_after[0] = (0, 0, 0)

num_sim_runs = 2500 # For part 2 we just need a reasonably long history of heights
# tower = create_empty_tower(-num_sim_runs*3)

def default_value(key):
    x, y = key

    if y >= 0: 
        return WALL_BLOCK
    if x < 0 or x > 6:
        return WALL_BLOCK

tower = keydefaultdict(default_value)
index_jets_command = 0
current_height = 0

for falling_rock_num in range(num_sim_runs): 
    rock = rock_types[falling_rock_num%len(rock_types)]
    rock = get_start_position(rock, tower)
    
    rock_stopped = False
    while not rock_stopped: 
        rock = move_sideways(rock, commands[index_jets_command % len(commands)], tower)
        index_jets_command += 1 
        rock, overlaps = move_downwards(rock, tower)
       
        # Stop the rocks if they would intersect
        if overlaps:
            rock_stopped = True
            for x, y in rock:
                tower[(x, y)] = True
                current_height = min(current_height, y)
    # assert current_height == height_max_block(tower)
    height_after[falling_rock_num+1] = (falling_rock_num%len(rock_types), index_jets_command % len(commands), abs(current_height))

    # Print info so I know my program is still running
    if falling_rock_num % 1000 == 0:
        print(falling_rock_num)        

print("part 1:", height_after[2022][2])

0
1000
2000
part 1: 3163


In [2]:
# We have a list with start-block, end-block, and heights... 
# We need to find the derivative of heights over a certain repeat range, 
# Keep in mind that the start of the repeat range can be different...
find_index = 1000000000000

# Since the heights will repeat at some point we can pick any start place
# as long as this start is inside the repeating pattern...
# I randomly picked something high enough and hoped for the best... 
start = 1000

loops_every = None
for repeat_range in range(20, 5000): # Just guessing the possible heights
    did_find = True
    in_loop_diff = list()

    # Check if this repeat range fits by checking if the height difference after one repeat is the same as the 
    # height difference after two repeats...
    for i in range(repeat_range):
        _, _, h1 = height_after[start+i]
        _, _, h2 = height_after[start+i+repeat_range]
        _, _, h3 = height_after[start+i+2*repeat_range]
        in_loop_diff.append(h1 - height_after[start][2])
        if h3 - h2 != h2 - h1:
            did_find = False
            break
    if did_find:
        loops_every = repeat_range
        print('found repeating', repeat_range)
        break
    
# Calculate the goal number    
num_loops = (find_index - start) // loops_every
height_diff_full_loop = height_after[start+loops_every][2] - height_after[start][2]
_, _, height_when_starts_looping = height_after[start]
answer = height_when_starts_looping + num_loops*height_diff_full_loop + in_loop_diff[(find_index - start) % loops_every]
print("part 2", answer)    

found repeating 1715
part 2 1560932944615


# Old code which did not work for me and cost me a lot of time

In [ ]:
# Find the loop
find_index = 1000000000000

seen_before = dict() # notes when it has seen something before
in_loop_diff = dict()

loops_after = None
height_when_starts_looping = None

debug_loop_index = list()
start_finding_loop = 2250
for i in range(len(height_after)): 
    index_rock, index_jet, ha = height_after[i]

    # TODO: Not only find a simple loop, but also something which actually repeats... 
    if (index_rock, index_jet) in seen_before:
        height_diff_full_loop = ha - seen_before[(index_rock, index_jet)][1][2]

        if i > start_finding_loop:
            print("LOOP", i, 'found repeat', (index_rock, index_jet), 'last time seen', seen_before[(index_rock, index_jet)])
            debug_loop_index.append(i)

        else: 
            continue
        if not loops_after and i > start_finding_loop:
            loops_after = seen_before[(index_rock, index_jet)][0]
            height_when_starts_looping = seen_before[(index_rock, index_jet)][1][2]
            loops_every = i - loops_after
            height_diff_full_loop = ha - seen_before[(index_rock, index_jet)][1][2]
            next_place = loops_after + loops_every
            if height_after[next_place][2] - height_when_starts_looping != height_diff_full_loop:
                # False alarm!
                loops_after = None
                continue
            else:
                print("ACTUAL LOOp", i)
        
        in_loop_diff[(i - loops_after - loops_every)] = ha - height_when_starts_looping - height_diff_full_loop
        # print("Some predictions", loops_after, loops_every, height_when_starts_looping, height_diff)
         
        #for i in range(10): 
        # loop_index = loops_after + i * loops_every
        # predicted_height = height_when_starts_looping + i * height_diff
        # print(loop_index, height_after[loop_index], 'predicted_height', predicted_height)
        
    else:
        seen_before[(index_rock, index_jet)] = (i, height_after[i])

num_loops = (find_index - loops_after) // loops_every
answer = height_when_starts_looping + num_loops*height_diff_full_loop + in_loop_diff[(find_index - loops_after) % loops_every]
print("part 2", answer)